In [1]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 163 kB in 2s (67.4 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state inf

## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="https://github.com/CGrannan/dsc-resilient-distributed-datasets-rdd-lab-online-ds-sp-000/blob/master/images/rdd_diagram.png?raw=1" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [9]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([9.05181856e-01, 4.98233135e-01, 2.24579120e+00, 3.23010347e+00,
       3.87251530e+00, 5.71798696e+00, 3.72605531e-01, 1.41358810e-01,
       2.34031510e+00, 4.14860000e+00, 1.27172552e+00, 9.42114273e-01,
       3.35252924e+00, 4.86032763e+00, 9.39288410e+00, 5.27535641e+00,
       1.01846013e+01, 3.24253227e+00, 6.01819640e+00, 1.58312762e+01,
       1.22630349e+01, 1.69593791e+01, 7.37787777e+00, 1.35777517e+01,
       2.78971030e+00, 1.36634510e+01, 1.72330163e+01, 1.09857415e+01,
       2.61854723e+00, 6.67607154e+00, 1.32249975e+01, 2.13501083e-01,
       6.75383161e+00, 1.92472643e+01, 4.82881340e+00, 1.47465888e+01,
       2.29509504e+01, 3.08232364e+01, 2.93548031e+01, 4.95137840e+00,
       1.58827537e+01, 1.78330669e+01, 2.34125705e+01, 4.21283650e+01,
       3.27391864e+01, 6.39132205e-01, 9.07109440e+00, 1.29144213e+00,
       3.94070881e+00, 4.17784041e-01, 6.30961472e+00, 4.25943989e+01,
       2.90865248e+00, 2.89606169e-01, 3.47355951e+01, 9.24588839e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(5)

[0.9051818560168843,
 0.49823313466107644,
 2.2457911982236687,
 3.2301034720241937,
 3.8725153015956497]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num*.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.8327673075355335,
 0.45837448388819035,
 2.0661279023657753,
 2.9716951942622583,
 3.562714077467998]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x: x*.9)

In [16]:
discounted.take(10)

[0.7494905767819802,
 0.4125370354993713,
 1.8595151121291977,
 2.6745256748360324,
 3.206442669721198,
 4.734493203202252,
 0.3085173796237339,
 0.1170450946989963,
 1.9377809033316868,
 3.4350407959264726]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x: x*.9).top(15)

[811.420811356118,
 807.394147046679,
 795.0643914855065,
 793.5336981330913,
 784.7028723488852,
 782.4372506318019,
 779.1151990962869,
 735.4466694038238,
 734.4001159603257,
 715.6519533096357,
 712.995608771232,
 712.4793570814628,
 709.3712989285401,
 707.7081013008252,
 686.2963329098093]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [18]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [19]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.9051818560168843, 0.7494905767819802), (0.49823313466107644, 0.4125370354993713), (2.2457911982236687, 1.8595151121291977), (3.2301034720241937, 2.6745256748360324), (3.8725153015956497, 3.206442669721198), (5.717986960389193, 4.734493203202252), (0.3726055309465385, 0.3085173796237339), (0.14135881002294237, 0.1170450946989963), (2.3403151006421337, 1.9377809033316868), (4.14859999508028, 3.4350407959264726)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [20]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.9051818560168843, 0.7494905767819802, 0.49823313466107644, 0.4125370354993713, 2.2457911982236687, 1.8595151121291977, 3.2301034720241937, 2.6745256748360324, 3.8725153015956497, 3.206442669721198]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [21]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

283

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [22]:
selected_items.reduce(lambda x, y: x + y)

132967.2094678756

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [23]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(3, 319.08468284204645),
 (3, 332.33136181855286),
 (45, 345.7049120534622),
 (5, 325.23139807669384),
 (25, 355.43229513004013),
 (21, 305.76644080342857),
 (26, 301.89665027692314)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [24]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x+y)
total_spent.take(10)

[(20, 1749.8991528079002),
 (30, 1860.4352291462105),
 (40, 1519.9343124413836),
 (10, 747.6502496175578),
 (50, 1140.1908091510327),
 (21, 3147.692150659921),
 (11, 1720.8591085063465),
 (41, 5530.551727296548),
 (1, 1886.7124644122694),
 (31, 2658.822761264671)]

In [25]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(42, 5863.905304820172),
 (41, 5530.551727296548),
 (18, 5345.575583482536),
 (44, 5317.971304055646),
 (3, 5247.657377060058),
 (17, 4625.55459195365),
 (13, 4323.797099563855),
 (27, 4275.552563070043),
 (45, 4163.407677461436),
 (7, 3701.678250494731),
 (22, 3669.345575855048),
 (2, 3445.8859283658658),
 (23, 3411.1049856442464),
 (39, 3398.0676150680783),
 (21, 3147.692150659921),
 (19, 3056.6017425972304),
 (25, 2958.8792079512696),
 (8, 2845.164347052575),
 (34, 2813.793552912673),
 (9, 2721.06235244923),
 (38, 2688.5111387348766),
 (31, 2658.822761264671),
 (12, 2642.4372152534997),
 (36, 2594.406765781414),
 (26, 2504.009276733129),
 (33, 2371.4683636907084),
 (47, 2335.6125971101606),
 (46, 2249.381505998731),
 (37, 2175.4508983209353),
 (16, 2104.5616124629582),
 (15, 2034.4932656091887),
 (35, 2003.2042236726938),
 (1, 1886.7124644122694),
 (30, 1860.4352291462105),
 (48, 1852.9884112060995),
 (28, 1797.7384084965743),
 (20, 1749.8991528079002),
 (11, 1720.8591085063465),
 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [26]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda x:x[1], reverse=True)

[(27, 12),
 (6, 12),
 (19, 11),
 (41, 10),
 (25, 10),
 (9, 10),
 (43, 10),
 (44, 9),
 (26, 9),
 (12, 9),
 (23, 8),
 (18, 8),
 (39, 8),
 (7, 7),
 (46, 7),
 (42, 7),
 (1, 7),
 (31, 6),
 (3, 6),
 (21, 6),
 (33, 6),
 (47, 6),
 (32, 6),
 (30, 6),
 (36, 6),
 (2, 5),
 (40, 5),
 (8, 5),
 (20, 5),
 (37, 4),
 (16, 4),
 (4, 4),
 (38, 4),
 (48, 4),
 (17, 4),
 (10, 4),
 (14, 4),
 (50, 3),
 (34, 3),
 (45, 3),
 (24, 3),
 (22, 2),
 (11, 2),
 (5, 2),
 (49, 2),
 (28, 2),
 (15, 2),
 (13, 2),
 (35, 2),
 (29, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
